# CENTHUD

In [1]:
%matplotlib inline
import seaborn as sns
import pandas as pd
import numpy as np


from icap.results.results import Results
from icap.database.icapdatabase import ICapDatabase

In [2]:
fp = 'icap/database/icapdatabase.json'
conn = ICapDatabase(fp).connect()

## CENTHUD Demand

In [ ]:
record_query = """select distinct
	RTrim(p.PremiseId) as PremiseId,
	RTrim(p.RateClass) as RateClass, 
    RTrim(p.Strata) as Strata,
	Year(m.EndDate) as Year,
	Avg(m.Demand) as AvgDemand
from [Premise] p
inner join [MonthlyUsage] m
	on m.PremiseId = p.PremiseId
where
	p.UtilityId = 'CENTHUD' and
	Month(m.EndDate) in (6, 7, 8) and
	m.Demand is not NULL
group by
	p.PremiseId, p.RateClass, p.Strata,
	Year(m.EndDate)"""

dmd_rec = pd.read_sql(record_query, conn)

In [ ]:
# only valid when [Premise] is omitted from join
dmd_rec[dmd_rec['PremiseId'] == '5616019800']

In [ ]:
load_profile_query = """select distinct
        RTrim(Strata) as Strata,
        Year(StartDate) - 1 as Year,
        ParameterValue
    from [UtilityParameterValue] upv
    where
        UtilityId = 'CENTHUD' and
        ParameterId like '%Weat%'"""

utility = pd.read_sql(load_profile_query, conn)

In [ ]:
tmp = pd.merge(dmd_rec, utility,
         how='outer',
         on=['Year', 'Strata'])

In [ ]:
tmp['ICap'] = tmp['AvgDemand'] * tmp['ParameterValue']

In [ ]:
tmp[tmp['PremiseId'] == '5616019800']

## CENTHUD (super)

In [3]:
from icap.centhud.centhud import CENTHUD
from icap.centhud.centhud import CENTHUDRecipe

In [4]:
c = CENTHUDRecipe(conn, Results).run_all()

In [5]:
# c.write_comparison_to_csv()
c.analyze_comparison(write_to_excel=True)
c.write_comparison_to_csv()

# c.compare_[c.compare_['PremiseId'] == '1222106000']
# c.compare_.dropna(subset=['HistoricalICap']).sort_values(by=['RecipeVariance'])

In [ ]:
c.analyze_comparison()

## CENTHUDConsumption

In [ ]:
from icap.centhud.centhud import CENTHUDConsumption
con = CENTHUDConsumption(conn)

In [ ]:
rec = con.records_.copy()
util = con.util_df_.copy()
sys = con.sys_df_.copy()

In [ ]:

tmp = pd.merge(rec, util,
        how='left',
        on=['Year', 'Strata'])


tmp_2 = pd.merge(tmp, sys,
                how='left', on=['Year'])



#pd.crosstab([tmp_2['Year'], tmp_2['Strata']], tmp_2['UtilityFactor'], dropna=False,margins=True)
tmp_2['ICap'] = tmp_2['LoadProfile'] * tmp_2['UsageFactor'] * tmp_2['UtilityFactor']
tmp_2.drop_duplicates(inplace=True)

In [ ]:
tmp_2[tmp_2['PremiseId'] == '1222106000']

In [ ]:
tmp_2.drop('RateClass_y', axis=1)
tmp_2.rename(columns={'RateClass_x':'RateClass'})

## CENTHUDDemand

In [ ]:
from icap.centhud.centhud import CENTHUDDemand
dmd = CENTHUDDemand(conn)

In [ ]:
icap = dmd.compute_icap().copy()

In [ ]:
#icap[icap['PremiseId'] == '1120030000']
icap[icap['PremiseId'] == '5616019800']

In [ ]:
rec = dmd.records_.copy()
util = dmd.util_df_.copy()
lp = dmd.load_profile.copy()

In [ ]:
rec = rec[rec['PremiseId'] == '1120030000'].copy()

In [ ]:
tmp = pd.merge(rec, util,
        how='left',
        on=['Year', 'RateClass', 'Strata'])

In [ ]:
tmp

In [ ]:
tmp = pd.merge(tmp, lp,
              how='left',
              on=['Strata', 'Year'])

In [ ]:
tmp

In [ ]:
tmp['ICap'] = tmp['AVGDemand'] * tmp['UtilFactor'] * tmp['AVGHourlyLoad']

# CONEDRecipe

In [ ]:
tmp

In [ ]:
from icap.results.results import Results
from icap.centhud.centhud import CENTHUDRecipe

In [ ]:
r = CENTHUDRecipe(conn, results=Results).run_all()

In [ ]:
r.compare_[r.compare_['PremiseId'] == '1120030000']